## Introduction

In this lab exercise, we are going to create a program that will break Diffie Hellman (DH) keys. The reason why we use Diffie Hellman and not RSA is that standard RSA requires a key-length of 1024 bits, which currently will require a lot of compute and time to break and not feasible for this lab.
The Diffie Hellman algorithm does not have this minimal requirement and "works" with smaller key-lengths. Smaller key-lengths "work" but that does not mean they are secure.

**Note**: Please add/edit your code in test.py and run it via a Terminal

### Alice & Bob
Alice and Bob would like to exchange a sensitive message message.
By generating a shared key using Diffie Hellman (DH) they are able to encrypt the sensitive message with their shared secret and prevent others from eaves dropping.

## General steps

Please follow these generic steps for each of the following sections.
* Copy the code into `test.py`
* This file has a reference to a library file `labfuncs.py` where the actual functions used are located
* Set a `bitSize` larger than 3
* Run the program: `python test.py` from a terminal in the lab1 directory


### Generate initial keys
There are specific requirements for DH thus we will write a program that will ensure this.

Alice will start the process:
* Alice generates `pubPrimeKey`, a public prime number and `pubBaseKey` a primitive root modulo of `pubPrimeKey`.


***NOTE***: Do not copy `def main():` line this is just there to prevent processing of the Jupyter Notebook

In [3]:
def main():

    bitSize = 4

    print("Given the bitSize of", bitSize)
    print("    the Private keys of Alice & Bob will be")
    print("         larger than:", pow(2, bitSize - 1),"and")
    print("        smaller than:", pow(2, bitSize) - 1)

    pubBaseKey = 0
    while (pubBaseKey < 2):
        pubPrimeKey = randPrime(bitSize)
        primitiveRoots, pubBaseKey = findPrimitiveRoot(pubPrimeKey)
    print("    the Public Prime key (p)")
    print("        is set to:", pubPrimeKey)
    print("    the derived Public Base key (g) is")
    print("        selected from:", primitiveRoots)
    print("        and set to:", pubBaseKey)


# Exchanging keys and generating private keys

* Alice shares `pubPrimeKey`, a prime number and `pubBaseKey` in the clear with Bob
* Alice and Bob both generate a personal secret key (`privA, privB`), based on the same `bitSize`. We make sure that `privA`, the secret key of Alice is not the same as `privB`, the secret key of Bob.
* Alice and Bob both calculate their public key (`pubA, pubB`)

***NOTE***: Do not copy `def main():` line this is just there to prevent processing of the Jupyter Notebook


In [4]:
def main():

    # ...

    privA = randN(bitSize)
    privB = privA
    while (privA == privB):
        privB = randN(bitSize)

    print("The Private key of")
    print("    Alice is:", privA)
    print("      Bob is:", privB)

    pubA = calcKey(privA, pubBaseKey, pubPrimeKey)
    pubB = calcKey(privB, pubBaseKey, pubPrimeKey)

    print("The Public key of")
    print("    Alice is:", pubA)
    print("      Bob is:", pubB)


## Calculating the shared Secret
In DH, the public and privates are *only* used to calculate the `sharedSecret`.
This `sharedSecret` is not communicated between Alice and Bob, but only used to encrypt messages using another algorithm like AES.
In RSA the public and private key can be used to exchange other messages, like a `sharedSecret` and thus is communicated over the path between Alice and Bob.

In [5]:
def main():

    # ...

    sharedSecretA = calcKey(privA, pubB, pubPrimeKey)
    sharedSecretB = calcKey(privB, pubA, pubPrimeKey)

    if (sharedSecretA != sharedSecretB):
        print("ERROR: Alice's & Bob's sharedSecret are not the same")
        exit()

    print("The Shared secret of")
    print("    Alice is:", sharedSecretA)
    print("      Bob is:", sharedSecretB)


## Introducing Mallory
Mallory has been able to listen in on the conversation between Alice and Bob.
She has been able to collect the following public information exchanged:
* Alice messages containing
  * `pubPrimeKey`
  * `pubBaseKey`
  * Alice's public key `pubA`
* Bob message containing
  * Bob's public key `pubB`

Malory will now try to calculate the private keys of Alice and Bob and their shared secret.

In [6]:
def main():

    # ...
    print("Malory determines")
    malPrivKeyA, malPrivKeyB = findSecretKey(pubA, pubB,
                                             pubPrimeKey, pubBaseKey)

    print("    Alice's private key to be:", malPrivKeyA)
    print("    Bob's private key to be:", malPrivKeyB)

    malSharedSecretA = calcKey(malPrivKeyA, pubB, pubPrimeKey)
    malSharedSecretB = calcKey(malPrivKeyB, pubA, pubPrimeKey)

    print("    Alice's shared secret to be:", malSharedSecretA)
    print("    Bob's shared secret to be:", malSharedSecretB)


## Malory's challenge
The function `findSecretKey` that Malory uses to find the private keys of Alice and Bob is quite time consuming.

    A = pubA
    B = pubB
    p = pubPrimeKey
    g = pubBaseKey

***QUESTION*** Which variable or number could you change to speed up the search of this function?

    def findSecretKey(A, B, p, g):
        a, b = None, None
        for x in range(1, p):
            if (g ** x) % p == A:
                a = x
            if (g ** x) % p == B:
                b = x
        return a, b

You are probably using small integers to perform these tasks and the process will become slower with `bitSize` increasing.
We need Quantum computation to speed up the process of this discrete logarithm problem.

Let's go to Lab2